# Compare Fiddler Models

This notebook provides comprehensive comparison capabilities for two Fiddler models, with support for:
* Cross-instance comparisons (compare models across different Fiddler deployments)
* Cross-model comparisons (compare any two models on same or different instances)
* Detailed configuration, schema, spec, segment, and custom metric comparison

## Use Cases
* **Version comparison:** Compare different versions of the same model
* **Cross-environment validation:** Verify consistency between dev/staging/prod
* **Migration planning:** Understand differences before migrating assets
* **Configuration audit:** Document and track model configuration changes

## Prerequisites
* Source Fiddler instance URL and API token
* Target Fiddler instance URL and API token (can be same as Source)
* Both models must exist

In [ ]:
%pip install -q fiddler-client

# Install fiddler_utils if not already available
import sys
sys.path.insert(0, '..')  # Add parent directory to path

In [ ]:
import fiddler as fdl
from fiddler_utils import (
    ModelComparator,
    ComparisonConfig,
    connection_context,
)

# Suppress verbose Fiddler client logs
# configure_fiddler_logging(level='ERROR')

print(f'Running Fiddler Python client version {fdl.__version__}')

## Configuration

In [ ]:
# Source Configuration
SOURCE_URL = ''  # e.g., 'https://company.fiddler.ai'
SOURCE_TOKEN = ''
SOURCE_PROJECT_NAME = ''
SOURCE_MODEL_NAME = ''
SOURCE_MODEL_VERSION = ''  # Optional, leave empty for unversioned models

# Target Configuration
TARGET_URL = ''  # e.g., 'https://company-dev.fiddler.ai'
TARGET_TOKEN = ''
TARGET_PROJECT_NAME = ''
TARGET_MODEL_NAME = ''
TARGET_MODEL_VERSION = ''  # Optional, leave empty for unversioned models

## Initialize Connections and Fetch Models

In [ ]:
# Fetch Source using connection_context
print("Connecting to Source Fiddler instance...")
with connection_context(SOURCE_URL, SOURCE_TOKEN, log_level="ERROR"):
    source_project = fdl.Project.from_name(SOURCE_PROJECT_NAME)
    source_model_kwargs = {
        'project_id': source_project.id,
        'name': SOURCE_MODEL_NAME
    }
    if SOURCE_MODEL_VERSION:
        source_model_kwargs['version'] = SOURCE_MODEL_VERSION
    
    source_model = fdl.Model.from_name(**source_model_kwargs)

print(f"Source: {source_model.name} (ID: {source_model.id})")
if hasattr(source_model, 'version') and source_model.version:
    print(f"  Version: {source_model.version}")

In [ ]:
# Fetch Target using connection_context
print("\nConnecting to Target Fiddler instance...")
with connection_context(TARGET_URL, TARGET_TOKEN, log_level="ERROR"):
    target_project = fdl.Project.from_name(TARGET_PROJECT_NAME)
    target_model_kwargs = {
        'project_id': target_project.id,
        'name': TARGET_MODEL_NAME
    }
    if TARGET_MODEL_VERSION:
        target_model_kwargs['version'] = TARGET_MODEL_VERSION
    
    target_model = fdl.Model.from_name(**target_model_kwargs)

print(f"Target: {target_model.name} (ID: {target_model.id})")
if hasattr(target_model, 'version') and target_model.version:
    print(f"  Version: {target_model.version}")

## Run Comprehensive Comparison

This cell uses `ModelComparator` from `fiddler_utils` to compare the models across all dimensions:
* Configuration (task, event ID column, timestamp column, task parameters)
* Schema (columns, roles, data types)
* Spec (inputs, outputs, targets, metadata, custom features)
* Segments
* Custom metrics
* Alerts
* Baselines
* Charts

### Using ComparisonConfig (Recommended)

You can use the convenient `ComparisonConfig` class to specify what to compare:

In [ ]:
# Create comparator and run comprehensive comparison
print("Creating ModelComparator...")
comparator = ModelComparator(source_model, target_model)

print("Running comprehensive comparison...\n")

# Option 1: Compare everything (default)
result = comparator.compare_all(config=ComparisonConfig.all())

# Option 2: Compare only schema
# result = comparator.compare_all(config=ComparisonConfig.schema_only())

# Option 3: Compare model structure without assets
# result = comparator.compare_all(config=ComparisonConfig.no_assets())

# Option 4: Custom configuration
# result = comparator.compare_all(
#     config=ComparisonConfig(
#         include_configuration=True,
#         include_schema=True,
#         include_spec=True,
#         include_segments=True,
#         include_custom_metrics=True,
#         include_alerts=False,
#         include_baselines=False,
#         include_charts=False,
#     )
# )

# Option 5: Old-style boolean parameters (still supported for backward compatibility)
# result = comparator.compare_all(
#     include_configuration=True,
#     include_schema=True,
#     include_spec=True,
#     include_segments=True,
#     include_custom_metrics=True,
#     include_alerts=True,
#     include_baselines=True,
#     include_charts=True,
# )

print("✓ Comparison complete!")
print(f"\nDifferences found: {result.has_differences()}")
print(f"Summary:\n {result.to_json()}")

## Display Formatted Comparison Report

In [ ]:
# Display the comparison report
print(result.to_markdown())

## Detailed Summary Statistics

Get detailed counts and breakdowns of differences:

In [ ]:
print("\n" + "="*70)
print("DETAILED COMPARISON SUMMARY")
print("="*70)

total_differences = 0

# Configuration differences
if result.configuration and result.configuration.has_differences():
    config_diffs = len(result.configuration.differences)
    print(f"\n⚠️  Configuration: {config_diffs} difference(s)")
    for key, diff in result.configuration.differences.items():
        print(f"    - {key}: {diff}")
    total_differences += config_diffs
else:
    print("\n✅ Configuration: Identical")

# Schema differences
if result.schema:
    schema_diffs = (
        len(result.schema.only_in_source) + 
        len(result.schema.only_in_target) + 
        len(result.schema.type_mismatches)
    )
    if schema_diffs > 0:
        print(f"\n⚠️  Schema: {schema_diffs} difference(s)")
        if result.schema.only_in_source:
            print(f"    - Only in Source: {len(result.schema.only_in_source)} columns")
        if result.schema.only_in_target:
            print(f"    - Only in Target: {len(result.schema.only_in_target)} columns")
        if result.schema.type_mismatches:
            print(f"    - Type mismatches: {len(result.schema.type_mismatches)} columns")
        total_differences += schema_diffs
    else:
        print("✅ Schema: Identical")

# Spec differences
if result.spec and result.spec.has_differences():
    print("\n⚠️  Spec: Differences detected")
    if not result.spec.inputs_match:
        print("    - Inputs differ")
        total_differences += 1
    if not result.spec.outputs_match:
        print("    - Outputs differ")
        total_differences += 1
    if not result.spec.targets_match:
        print("    - Targets differ")
        total_differences += 1
    if not result.spec.metadata_match:
        print("    - Metadata differ")
        total_differences += 1
    if not result.spec.decisions_match:
        print("    - Decisions differ")
        total_differences += 1
    if not result.spec.custom_features_match:
        print("    - Custom features differ")
        total_differences += 1
else:
    print("✅ Spec: Identical")

# Segments
if result.segments and result.segments.has_differences():
    seg_diffs = result.segments.total_differences
    print(f"\n⚠️  Segments: {seg_diffs} difference(s)")
    if result.segments.only_in_source:
        print(f"    - Only in Source: {len(result.segments.only_in_source)}")
    if result.segments.only_in_target:
        print(f"    - Only in Target: {len(result.segments.only_in_target)}")
    if result.segments.definition_differences:
        print(f"    - Different definitions: {len(result.segments.definition_differences)}")
    total_differences += seg_diffs
else:
    print("✅ Segments: Identical")

# Custom Metrics
if result.custom_metrics and result.custom_metrics.has_differences():
    metric_diffs = result.custom_metrics.total_differences
    print(f"\n⚠️  Custom Metrics: {metric_diffs} difference(s)")
    if result.custom_metrics.only_in_source:
        print(f"    - Only in Source: {len(result.custom_metrics.only_in_source)}")
    if result.custom_metrics.only_in_target:
        print(f"    - Only in Target: {len(result.custom_metrics.only_in_target)}")
    if result.custom_metrics.definition_differences:
        print(f"    - Different definitions: {len(result.custom_metrics.definition_differences)}")
    total_differences += metric_diffs
else:
    print("✅ Custom Metrics: Identical")

# Alerts
if result.alerts and result.alerts.has_differences():
    alert_diffs = result.alerts.total_differences
    print(f"\n⚠️  Alerts: {alert_diffs} difference(s)")
    if result.alerts.only_in_source:
        print(f"    - Only in Source: {len(result.alerts.only_in_source)}")
    if result.alerts.only_in_target:
        print(f"    - Only in Target: {len(result.alerts.only_in_target)}")
    total_differences += alert_diffs
else:
    print("✅ Alerts: Identical")

# Baselines
if result.baselines and result.baselines.has_differences():
    baseline_diffs = result.baselines.total_differences
    print(f"\n⚠️  Baselines: {baseline_diffs} difference(s)")
    if result.baselines.only_in_source:
        print(f"    - Only in Source: {len(result.baselines.only_in_source)}")
    if result.baselines.only_in_target:
        print(f"    - Only in Target: {len(result.baselines.only_in_target)}")
    total_differences += baseline_diffs
else:
    print("✅ Baselines: Identical")

# Charts (note: limited comparison as charts are project-level)
if result.charts and result.charts.has_differences():
    chart_diffs = result.charts.total_differences
    print(f"\n⚠️  Charts: {chart_diffs} difference(s)")
    total_differences += chart_diffs
else:
    print("✅ Charts: Identical (or project-level asset)")

# Overall assessment
print("\n" + "-"*70)
if total_differences == 0:
    print("\n🎉 OVERALL: Models are IDENTICAL across all compared dimensions")
else:
    print(f"\n📊 OVERALL: {total_differences} total difference(s) detected")
    print("\nRecommendation: Review the detailed comparison report above to")
    print("understand the specific differences between the two models.")

print("\n" + "="*70)

## Access Individual Comparison Results

You can access detailed comparison results programmatically:

In [ ]:
# Example: List all segments only in Source
if result.segments and result.segments.only_in_source:
    print("Segments only in Source:")
    for segment_name in result.segments.only_in_source:
        print(f"  - {segment_name}")

# Example: Show custom metrics with different definitions
if result.custom_metrics and result.custom_metrics.definition_differences:
    print("\nCustom metrics with different definitions:")
    for metric_name, diff in result.custom_metrics.definition_differences.items():
        print(f"  - {metric_name}:")
        print(f"      {diff}")

# Example: Check schema type mismatches
if result.schema and result.schema.type_mismatches:
    print("\nSchema type mismatches:")
    for col_name, (type_a, type_b) in result.schema.type_mismatches.items():
        print(f"  - {col_name}: {type_a} (Source) vs {type_b} (Target)")

if result.schema and result.schema.only_in_source:
    print("\nColumns only in Source:")
    for col_name in result.schema.only_in_source:
        print(f"  - {col_name}")

if result.schema and result.schema.only_in_target:
    print("\nColumns only in Target:")
    for col_name in result.schema.only_in_target:
        print(f"  - {col_name}")